In [1]:
!pip install tabula-py
import tabula
import pandas as pd
import numpy as np

In [2]:
table = tabula.read_pdf("G1.pdf", pages=1)

In [3]:
for i in range(len(table)):
    table[i].columns = table[i].columns.str.replace(r"\r", " ")

In [4]:
table[0] = table[0].drop(columns=["Unnamed: 0"])

In [5]:
index = table[0].index
condition1 = table[0]['CHICOPEE BIG Y'] == "WEEKDAY"
condition2 = table[0]['CHICOPEE BIG Y'] == 'SATURDAY'
condition3 = table[0]['CHICOPEE BIG Y'] == 'SUNDAY'
day_indices = index[condition1][0], index[condition2][0], index[condition3][0]

table_weekday = table[0].iloc[day_indices[0] + 1: day_indices[1] - 1]
table_saturday = table[0].iloc[day_indices[1] + 1: day_indices[2] - 1]
table_sunday = table[0].iloc[day_indices[2] + 1: len(table[0])]
table_weekday.reset_index(drop=True, inplace=True)
table_saturday.reset_index(drop=True, inplace=True)
table_sunday.reset_index(drop=True, inplace=True)

In [6]:
def preprocess_table(table):
    for i in range(len(table.columns)):
        col = table.columns[i]
        table[col] = table[col].str.replace('[\|A|B|C|D|E|F|G|H|I|J|K|L|M|N|O|P|Q|R|S|T|U|V|W|X|Y|Z|-]','')

In [7]:
preprocess_table(table_weekday)
preprocess_table(table_saturday)
preprocess_table(table_sunday)

C:\Users\quynh\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [8]:
# condition on mornings for weekdays. 
condition_weekday_morning = table_weekday['CHICOPEE BIG Y'].str.contains('[1][2]:[0-5][0-9]').idxmax()
table_weekday_morning = table_weekday.iloc[0: condition_weekday_morning]
table_weekday_morning.reset_index(drop=True, inplace=True)

In [9]:
# condition on evenings for weekdays
condition_weekday_evening = table_weekday['CHICOPEE BIG Y.1'].str.contains('[1][2]:[0-5][0-9]').idxmax()
table_weekday_evening = table_weekday.iloc[condition_weekday_evening - 1:]
table_weekday_evening.reset_index(drop=True, inplace=True)

In [10]:
# condition on mornings for saturdays. 
condition_saturday_morning = table_saturday['CHICOPEE BIG Y'].str.contains('[1][2]:[0-5][0-9]').idxmax()
table_saturday_morning = table_saturday.iloc[0: condition_saturday_morning]
table_saturday_morning.reset_index(drop=True, inplace=True)

In [11]:
# condition on evenings for saturdays. 
condition_saturday_evening = table_saturday['CHICOPEE BIG Y.1'].str.contains('[1][2]:[0-5][0-9]').idxmax()
table_saturday_evening = table_saturday.iloc[condition_saturday_evening - 1:]
table_saturday_evening.reset_index(drop=True, inplace=True)

In [12]:
# condition on mornings for sunday. 
condition_sunday_morning = table_sunday['CHICOPEE BIG Y'].str.contains('[1][2]:[0-5][0-9]').idxmax()
table_sunday_morning = table_sunday.iloc[0: condition_sunday_morning]
table_sunday_morning.reset_index(drop=True, inplace=True)

In [13]:
# condition on evenings for sunday. 
condition_sunday_evening = table_sunday['CHICOPEE BIG Y.1'].str.contains('[1][2]:[0-5][0-9]').idxmax()
table_sunday_evening = table_sunday.iloc[condition_sunday_evening - 1:]
table_sunday_evening.reset_index(drop=True, inplace=True)

In [14]:
def preprocess_times_morning(table):
    for i in range(len(table.columns)):
        if i != 0:
            col = table.columns[i]
            index = table[col].str.contains('[1][2]:[0-5][0-9]').idxmax()
            table.loc[index:,col] = ''

In [15]:
def preprocess_times_evening(table):
    for i in range(len(table.columns)):
        col = table.columns[i]
        index = table[col].str.contains('[1][2]:[0-5][0-9]').idxmax()
        if index != -1:
            table.loc[0:index-1,col] = ''

In [16]:
table_weekday_evening = table_weekday_evening.copy()
preprocess_times_evening(table_weekday_evening)
table_saturday_evening = table_saturday_evening.copy()
preprocess_times_evening(table_saturday_evening)
table_sunday_evening = table_sunday_evening.copy()
preprocess_times_evening(table_sunday_evening)
table_weekday_morning = table_weekday_morning.copy()
preprocess_times_morning(table_weekday_morning)
table_saturday_morning = table_saturday_morning.copy()
preprocess_times_morning(table_saturday_morning)
table_sunday_morning = table_sunday_morning.copy()
preprocess_times_morning(table_sunday_morning)

In [17]:
table_weekday_morning = table_weekday_morning.replace(r'^\s*$', np.nan, regex=True)
table_weekday_evening = table_weekday_evening.replace(r'^\s*$', np.nan, regex=True)
table_saturday_morning = table_saturday_morning.replace(r'^\s*$', np.nan, regex=True)
table_saturday_evening = table_saturday_evening.replace(r'^\s*$', np.nan, regex=True)
table_sunday_morning = table_sunday_morning.replace(r'^\s*$', np.nan, regex=True)
table_sunday_evening = table_sunday_evening.replace(r'^\s*$', np.nan, regex=True)

# Drop apply
table_weekday_morning = table_weekday_morning.apply(lambda x: pd.Series(x.dropna().values)).fillna('')
table_weekday_evening = table_weekday_evening.apply(lambda x: pd.Series(x.dropna().values)).fillna('')
table_saturday_morning = table_saturday_morning.apply(lambda x: pd.Series(x.dropna().values)).fillna('')
table_saturday_evening = table_saturday_evening.apply(lambda x: pd.Series(x.dropna().values)).fillna('')
table_sunday_morning = table_sunday_morning.apply(lambda x: pd.Series(x.dropna().values)).fillna('')
table_sunday_evening = table_sunday_evening.apply(lambda x: pd.Series(x.dropna().values)).fillna('')

In [20]:
table_weekday_morning.to_csv(r'weekday_morning.csv', index=False, header=True) 
table_weekday_evening.to_csv(r'weekday_evening.csv', index=False, header=True) 
table_saturday_morning.to_csv(r'saturday_morning.csv', index=False, header=True) 
table_saturday_evening.to_csv(r'saturday_evening.csv', index=False, header=True) 
table_sunday_morning.to_csv(r'sunday_morning.csv', index=False, header=True) 
table_sunday_evening.to_csv(r'sunday_evening.csv', index=False, header=True) 